In [1]:
# Importar librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split # para entrenar al modelo
from sklearn.preprocessing import StandardScaler 

In [2]:
# Ajustar límites
pd.options.display.max_columns = 78
pd.options.display.max_rows = 78

In [3]:
# DF
Data = pd.read_csv('Berlin_76_Final.csv')
Berlin = Data.drop(['Unnamed: 0'], axis=1)
Berlin

,listing_url,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,bathrooms_text,amenities,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,https://www.airbnb.com/rooms/3176,2025-06-21,city scrape,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,prenzlauer berg,https://a0.muscache.com/pictures/airflow/Hosti...,3718,https://www.airbnb.com/users/show/3718,Britta,2008-10-19,"Coledale, Australia",We love to travel ourselves a lot and prefer t...,within a few hours,f,https://a0.muscache.com/im/users/3718/profile_...,https://a0.muscache.com/im/users/3718/profile_...,prenzlauer berg,"['email', 'phone']",t,t,"Berlin, Germany",prenzlauer berg südwest,Pankow,52.534710,13.418100,Entire rental unit,Entire home/apt,1 bath,"[""Iron"", ""Cleaning products"", ""Wifi"", ""Oven"", ...",t,2025-06-21,2009-06-20,2025-03-02,No License,f,100.000000,80.000000,1.0,1.0,2.0,1.0,1.0,2.0,105.000000,63.0,730.0,63.0,63.0,730.0,730.0,63.0,730.0,0,4,4,250,149.0,1.0,0.0,79,0.0,126,13230.0,4.63,4.67,4.52,4.65,4.70,4.92,4.61,1.0,1.0,0.0,0.0,0.76
1,https://www.airbnb.com/rooms/9991,2025-06-21,city scrape,Geourgeous flat - outstanding views,4 bedroom with very large windows and outstand...,prenzlauer berg,https://a0.muscache.com/pictures/42799131/59c8...,33852,https://www.airbnb.com/users/show/33852,Philipp,2009-08-25,"Berlin, Germany",Born in Munich - lives in and loves Berlin. Wo...,within a day,f,https://a0.muscache.com/im/users/33852/profile...,https://a0.muscache.com/im/users/33852/profile...,prenzlauer berg,"['email', 'phone']",t,t,"Berlin, Germany",prenzlauer berg südwest,Pankow,52.532690,13.418050,Entire rental unit,Entire home/apt,2.5 baths,"[""Iron"", ""Wifi"", ""Babysitter recommendations"",...",t,2025-06-21,2015-08-09,2020-01-04,03/Z/RA/003410-18,f,50.000000,87.618712,1.0,1.0,7.0,2.5,1.0,4.0,135.000000,6.0,14.0,6.0,6.0,14.0,14.0,6.0,14.0,0,0,5,193,7.0,0.0,0.0,29,0.0,0,0.0,5.00,5.00,5.00,5.00,5.00,4.86,4.86,1.0,1.0,0.0,0.0,0.06
2,https://www.airbnb.com/rooms/14325,2025-06-21,city scrape,Studio Apartment in Prenzlauer Berg,The apartment is located on the upper second f...,prenzlauer berg,https://a0.muscache.com/pictures/508703/24988a...,55531,https://www.airbnb.com/users/show/55531,Chris + Oliver,2009-11-18,"Berlin, Germany","From Berlin, Germany. We love to travel!",within a day,t,https://a0.muscache.com/im/users/55531/profile...,https://a0.muscache.com/im/users/55531/profile...,prenzlauer berg,"['email', 'phone', 'work_email']",t,t,prenzlauer berg nordwest,prenzlauer berg nordwest,Pankow,52.548130,13.403660,Entire rental unit,Entire home/apt,1 bath,"[""Hangers"", ""Wifi"", ""Coffee maker"", ""Host gree...",t,2025-06-21,2010-06-29,2023-11-30,No License,f,100.000000,25.000000,4.0,5.0,1.0,1.0,0.0,1.0,75

In [4]:
# Formato de variables
Berlin['host_id'] = Berlin['host_id'].astype(str)
Berlin['latitude'] = Berlin['latitude'].astype(str)
Berlin['longitude'] = Berlin['longitude'].astype(str)
Berlin['host_is_superhost_num'] = Berlin['host_is_superhost'].map({'t':1,'f':0})
Berlin['host_has_profile_pic_num'] = Berlin['host_has_profile_pic'].map({'t':1,'f':0})
Berlin['host_identity_verified_num'] = Berlin['host_identity_verified'].map({'t':1,'f':0})
Berlin['has_availability_num'] = Berlin['has_availability'].map({'t':1,'f':0})
Berlin['instant_bookable_num'] = Berlin['instant_bookable'].map({'t':1,'f':0})

In [5]:
# Selección de variables
Berlin.head()

,listing_url,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,bathrooms_text,amenities,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable,host_response_rate,host_acceptance_rate,host_listings_count,...,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_is_superhost_num,host_has_profile_pic_num,host_identity_verified_num,has_availability_num,instant_bookable_num
0,https://www.airbnb.com/rooms/3176,2025-06-21,city scrape,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,prenzlauer berg,https://a0.muscache.com/pictures/airflow/Hosti...,3718,https://www.airbnb.com/users/show/3718,Britta,2008-10-19,"Coledale, Australia",We love to travel ourselves a lot and prefer t...,within a few hours,f,https://a0.muscache.com/im/users/3718/profile_...,https://a0.muscache.com/im/users/3718/profile_...,prenzlauer berg,"['email', 'phone']",t,t,"Berlin, Germany",prenzlauer berg südwest,Pankow,52.53471,13.4181,Entire rental unit,Entire home/apt,1 bath,"[""Iron"", ""Cleaning products"", ""Wifi"", ""Oven"", ...",t,2025-06-21,2009-06-20,2025-03-02,No License,f,100.0,80.000000,1.0,...,1.0,2.0,105.000000,63.0,730.0,63.0,63.0,730.0,730.0,63.0,730.0,0,4,4,250,149.0,1.0,0.0,79,0.0,126,13230.0,4.63,4.67,4.52,4.65,4.70,4.92,4.61,1.0,1.0,0.0,0.0,0.76,0,1,1,1,0
1,https://www.airbnb.com/rooms/9991,2025-06-21,city scrape,Geourgeous flat - outstanding views,4 bedroom with very large windows and outstand...,prenzlauer berg,https://a0.muscache.com/pictures/42799131/59c8...,33852,https://www.airbnb.com/users/show/33852,Philipp,2009-08-25,"Berlin, Germany",Born in Munich - lives in and loves Berlin. Wo...,within a day,f,https://a0.muscache.com/im/users/33852/profile...,https://a0.muscache.com/im/users/33852/profile...,prenzlauer berg,"['email', 'phone']",t,t,"Berlin, Germany",prenzlauer berg südwest,Pankow,52.53269,13.41805,Entire rental unit,Entire home/apt,2.5 baths,"[""Iron"", ""Wifi"", ""Babysitter recommendations"",...",t,2025-06-21,2015-08-09,2020-01-04,03/Z/RA/003410-18,f,50.0,87.618712,1.0,...,1.0,4.0,135.000000,6.0,14.0,6.0,6.0,14.0,14.0,6.0,14.0,0,0,5,193,7.0,0.0,0.0,29,0.0,0,0.0,5.00,5.00,5.00,5.00,5.00,4.86,4.86,1.0,1.0,0.0,0.0,0.06,0,1,1,1,0
2,https://www.airbnb.com/rooms/14325,2025-06-21,city scrape,Studio Apartment in Prenzlauer Berg,The apartment is located on the upper second f...,prenzlauer berg,https://a0.muscache.com/pictures/508703/24988a...,55531,https://www.airbnb.com/users/show/55531,Chris + Oliver,2009-11-18,"Berlin, Germany","From Berlin, Germany. We love to travel!",within a day,t,https://a0.muscache.com/im/users/55531/profile...,https://a0.muscache.com/im/users/55531/profile...,prenzlauer berg,"['email', 'phone', 'work_email']",t,t,prenzlauer berg nordwest,prenzlauer berg nordwest,Pankow,52.54813,13.40366,Entire rental unit,Entire home/apt,1 bath,"[""Hangers"", ""Wifi"", ""Coffee maker"", ""Host gree...",t,2025-06-21,2010-06-29,2023-11-30,No

In [6]:
# Identificar variables dicotómicas (exactamente 2 valores únicos)
dicotomicas = [col for col in Berlin.columns if Berlin[col].nunique() == 2]

print("Variables dicotómicas encontradas:")
print(dicotomicas)

Variables dicotómicas encontradas:
['last_scraped', 'source', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'calendar_last_scraped', 'instant_bookable', 'host_is_superhost_num', 'host_has_profile_pic_num', 'host_identity_verified_num', 'has_availability_num', 'instant_bookable_num']


In [7]:
# Tabla de correlaciones
Berlin_cuant = Berlin.select_dtypes(['int64','float64'])
Berlin_cuant.corr()

,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_is_superhost_num,host_has_profile_pic_num,host_identity_verified_num,has_availability_num,instant_bookable_num
host_response_rate,1.000000,0.339578,-0.084186,-0.080148,0.112577,0.014980,0.026643,0.122026,0.023193,-0.271874,-0.149060,-0.272246,-0.234988,-0.070802,-0.034255,-0.271758,-0.048368,0.060163,0.091804,0.110838,0.137775,0.212647,0.278294,0.262167,0.142788,0.250125,0.315556,0.273274,0.142961,0.142637,0.151628,0.145929,0.145625,0.143113,0.142979,0.087016,0.105067,0.071319,0.000326,0.287392,0.297236,0.021942,0.100784,0.106924,0.143397
host_acceptance_rate,0.339578,1.000000,-0.054875,-0.064361,0.087715,-0.020094,-0.007619,0.075098,0.095858,-0.010704,0.101183,-0.011635,-0.029771,0.183056,0.217152,-0.010998,0.201304,0.006273,0.013906,0.016840,0.022688,0.170200,0.206673,0.160895,0.031120,0.185410,0.178461,0.186516,0.032657,0.037654,0.042346,0.040171,0.036138,0.038989,0.034430,-0.137396,0.031989,0.077098,-0.022923,0.208719,0.147663,0.008032,-0.021926,-0.019888,0.337470
host_listings_count,-0.084186,-0.054875,1.000000,0.996923,0.000632,-0.026034,-0.008208,-0.023027,0.003843,0.134396,0.109663,0.141520,-0.027988,0.062675,0.053485,0.149187,0.055485,0.038026,0.030559,0.029001,0.104598,-0.099205,-0.096612,-0.094087,0.061546,-0.090912,-0.111379,-0.076091,-0.305447,-0.304101,-0.300663,-0.305370,-0.307242,-0.301347,-0.310397,0.444451,0.210400,-0.030248,-0.009285,-0.114915,-0.101279,0.042474,0.067819,0.054819,0.214558
host_total_listings_count,-0.080148,-0.064361,0.996923,1.000000,0.004504,-0.024556,-0.006016,-0.017418,0.001976,0.128088,0.101570,0.135026,-0.029830,0.054981,0.046309,0.142577,0.048009,0.048491,0.041566,0.040567,0.114275,-0.098889,-0.095731,-0.093018,0.073526,-0.090546,-0.109934,-0.074484,-0.304083,-0.302761,-0.299378,-0.303898,-0.305681,-0.299485,-0.309099,0.461284,0.223158,-0.030856,-0.001150,-0.113348,-0.103374,0.044981,0.071032,0.057445,0.212650
accommodates,0.112577,0.087715,0.000632,0.004504,1.000000,0.264063,0.559806,0.704707,0.288068,-0.189371,-0.041780,-0.191448,-0.186684,-0.017855,0.002982,-0.191333,-0.001937,0.033779,0.049411,0.070357,0.123448,0.128345,0.170865,0.128767,0.115367,0.153233,0.151767,0.302208,0.067414,0.067336,0.069014,0.069961,0.068645,0.068485,0.065303,0.053226,0.102046,-0.089870,-0.022803,0.169680,0.059736,0.000406,0.066122,0.068234,0.124746
bathrooms,0.014980,-0.020094,-0.026034,-0.024556,0.264063,1.000000,0.326673,0.235728,0.197652,-0.044623,-0.027922,-0.043648,-0.044365,-0.039424,-0.036274,-0.045507,-0.034944,-0.001601,0.004321,0.005822,0.031717,0.012353,0.017430,0.003537,0.022160,0.022662,-0.006858,0.105546,0.014193,0.011792,0.011887,0.010289,0.012955,0.009636,0.011973,-0.024670,-0.004662,-0.038382,-0.016748,0.000644,0.034386,-0.002099,0.035489,0.027117,-0.027825
bedrooms,0.026643,-0.007619,-0.008208,-0.006016,0.559806,0.326673,1.000000,0.495652,0.252244,-0.045644,0.002009,-0.045787,-0.042166,-0.009404,0.002950,-0.044771,0.000743,-0.039817,-0.036108,-0.027514,0.002738,0.012533,0.020460,0.015731,-0.006565,0.015701,-0.002412,0.174629,0.008197,0.006035,0.007278,0.006564,0.008569,0.006350,0.008248,-0.013123,0.007509,-0.101820,-0.018454,0.026144,-0.00